# Pinetree simulation example

https://pinetree.readthedocs.io/en/latest/intro.html#construct-a-simulation

In [ ]:
!pip install cmake
!pip install pinetree

In [ ]:
import pinetree as pt

model = pt.Model(cell_volume=8e-16)

In [ ]:
# Pinetree supports linear genomes of any size, represented by Genome objects. 
# A Genome object must be given a name and a length, in base pairs.
plasmid = pt.Genome(name="myplasmid", length=300)

In [ ]:
plasmid.add_promoter(name="phi1", start=1, stop=10,
                     interactions={"rnapol": 2e8})

plasmid.add_terminator(name="t1", start=299, stop=300,
                       efficiency={"rnapol": 1.0})

plasmid.add_gene(name="rnapol", start=26, stop=225,
                 rbs_start=11, rbs_stop=26, rbs_strength=1e7)

plasmid.add_gene(name="proteinX", start=241, stop=280,
                 rbs_start=226, rbs_stop=241, rbs_strength=1e7)

In [ ]:
# When all genetic elements have been added, register the Genome object with our Model object.
model.register_genome(plasmid)

In [ ]:
# To simulate both transcription and translation, we’ll add polymerases and then add ribosomes. 
# Since these enzymes may interact with more than one type of genome, we add them to the Model object.

model.add_polymerase(name="rnapol", footprint=10, speed=40.0, copy_number=10)
model.add_ribosome(speed=30.0, footprint=10, copy_number=100)

In [ ]:
# Pinetree supports option species reactions between one or two molecular species.
# For example, we may define a reaction such that proteinX forms a complex with rnapol called rnapol-X.

model.add_reaction(reactants=['proteinX', 'rnapol'],
                   products=['rnapol-X'],
                   rate_constant=1e-7)

In [ ]:
model.simulate(time_limit=60, time_step=1, output="my_simulation.tsv")

A pinetree simulation produces an output file with 5 columns.

time
    
    Current time of simulation in seconds.

species
    
    Name of a molecular species, derived from a polymerase name, a gene name, or an explicitly defined molecular species. Any name with a ‘__’ double underscore prefix is used internally by pinetree.

protein
    
    Quantity of free proteins corresponding to a species name. For example, the number in this column corresponding to ‘rnapol’ would represent free RNA polymerases that are not actively transcribing.

transcript
    
    Quantity of transcripts for corresponding to a species name. If a species only exists as a protein or otherwise has no transcript precursor, this value will be 0.

ribo_density (experimental)
    
    Average quantity of ribosomes actively translating on a transcript. 